<a href="https://colab.research.google.com/github/KaabiTaoufik/ImageProcessing/blob/main/ImageProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SyntaxError: ignored